# Experiments with Home Credit

In [44]:
import time
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from lightgbm import LGBMClassifier
from sklearn.inspection import PartialDependenceDisplay
from lime import lime_tabular
import shap
from cfmining.algorithms import MAPOCAM, BruteForce, Greedy
from cfmining.criteria import PercentileCalculator, PercentileCriterion, PercentileChangesCriterion, NonDomCriterion
from cfmining.predictors import MonotoneClassifier
from cfmining.visualization import buildTable, PlotCounterfactuals
from cfmining.mip_builder import RecourseBuilder
from cfmining.action_set import ActionSet
import joblib

from aif360.algorithms.preprocessing import Reweighing
from aif360.datasets import BinaryLabelDataset
from fairgbm import FairGBMClassifier
from sklego.linear_model import EqualOpportunityClassifier, DemographicParityClassifier
from fairlearn.postprocessing import ThresholdOptimizer

import plotly.express as px
import plotly.graph_objects as go


import credit_pipeline.data_exploration as dex
from credit_pipeline.training import *


%load_ext autoreload
%autoreload 2

OSError: /lib/x86_64-linux-gnu/libm.so.6: version `GLIBC_2.29' not found (required by /home/jansen/anaconda3/envs/german2/lib/python3.8/site-packages/fairgbm/lib_lightgbm.so)

## Loading and Cleaning

In [2]:
seed_number = 0

In [4]:
path = ""
df = dex.read_csv_encoded(path, 'application_train.csv')

In [5]:
columns_to_drop = dex.check_missing(df, 50,  False)
columns_to_drop

['COMMONAREA_MEDI',
 'COMMONAREA_AVG',
 'COMMONAREA_MODE',
 'NONLIVINGAPARTMENTS_MODE',
 'NONLIVINGAPARTMENTS_AVG',
 'NONLIVINGAPARTMENTS_MEDI',
 'FONDKAPREMONT_MODE',
 'LIVINGAPARTMENTS_MODE',
 'LIVINGAPARTMENTS_AVG',
 'LIVINGAPARTMENTS_MEDI',
 'FLOORSMIN_AVG',
 'FLOORSMIN_MODE',
 'FLOORSMIN_MEDI',
 'YEARS_BUILD_MEDI',
 'YEARS_BUILD_MODE',
 'YEARS_BUILD_AVG',
 'OWN_CAR_AGE',
 'LANDAREA_MEDI',
 'LANDAREA_MODE',
 'LANDAREA_AVG',
 'BASEMENTAREA_MEDI',
 'BASEMENTAREA_AVG',
 'BASEMENTAREA_MODE',
 'EXT_SOURCE_1',
 'NONLIVINGAREA_MODE',
 'NONLIVINGAREA_AVG',
 'NONLIVINGAREA_MEDI',
 'ELEVATORS_MEDI',
 'ELEVATORS_AVG',
 'ELEVATORS_MODE',
 'WALLSMATERIAL_MODE',
 'APARTMENTS_MEDI',
 'APARTMENTS_AVG',
 'APARTMENTS_MODE',
 'ENTRANCES_MEDI',
 'ENTRANCES_AVG',
 'ENTRANCES_MODE',
 'LIVINGAREA_AVG',
 'LIVINGAREA_MODE',
 'LIVINGAREA_MEDI',
 'HOUSETYPE_MODE']

In [6]:
df = df.drop(columns_to_drop, axis=1)

In [7]:
df_cols = df.columns.to_list()
obj_cols = dex.list_by_type(df, ['O'])

In [8]:
def days_to_years(dataframe, col_name):
        """
        Converts values from string to numeric.
        Uses the map function to convert the information on days employed to years employed
        """
        df_name = dataframe.copy()

        if col_name in df.columns:
            #Converts values from string to numeric.
            df_name[col_name] = pd.to_numeric(df_name[col_name], errors='coerce')

            #drops null values on the column
            df_name = df_name.dropna(subset=[col_name])

            #Use the map function to convert the information on days employed to years employed
            year = df_name.loc[:, col_name].map(lambda x: int(abs(x / 365)), na_action=None)
            df_name['YEARS'+col_name[4:]] = year

            #drops the column
            df_name = df_name.drop(col_name, axis=1)

        return df_name


df = days_to_years(df, "DAYS_EMPLOYED")
df = days_to_years(df, 'DAYS_BIRTH')

## Training Basic Models

In [9]:
X_acp = df.iloc[:, (df.columns != "TARGET") & (df.columns != "SK_ID_CURR")]
y_acp = df["TARGET"]

In [18]:
X_train, X_test, y_train, y_test = train_test_split(
    X_acp, 
    y_acp, 
    test_size = 0.2, 
    random_state=seed_number, 
    stratify=y_acp
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, 
    y_train, 
    test_size = 0.2, 
    random_state=seed_number, 
    stratify=y_train
)
A_train = (X_train.CODE_GENDER  == "F").astype(int)
A_val = (X_val.CODE_GENDER  == "F").astype(int)
A_test = (X_test.CODE_GENDER  == "F").astype(int)

In [11]:
classifiers = {
    "Logistic Regression": LogisticRegression,
    "Random Forest": RandomForestClassifier,
    "LightGBM": LGBMClassifier,
    "MLPC" : MLPClassifier,
}

In [12]:
param_spaces = {
    "LogisticRegression": {
        'C': {'low': 0.001, 'high': 10, 'log': True, 'type':'float'},
        'max_iter': {'low': 1000, 'high': 1000, 'step':1, 'type':'int'},
        'penalty': {'choices': ['l2'], 'type':'categorical'},
        "class_weight" : {"choices" : ["balanced"], 'type':'categorical'},
    },
    "RandomForestClassifier": {
        'n_estimators': {'low':10, 'high':150, 'step':20, 'type':'int'},
        'max_depth': {'low':2, 'high':10, 'type':'int'},
        'criterion': {'choices':['gini', 'entropy'], 'type':'categorical'},
        'min_samples_leaf' : {"low" : 1, "high" : 51, "step" : 5, 'type':'int'},
        "max_features" : {"low" : 0.1, "high" : 1.0, "type" : "float"},
        "class_weight" : {"choices" : ["balanced"], 'type':'categorical'},
    },
    "LGBMClassifier": {
        'learning_rate': {'low': 0.01, 'high': 1.0, 'type': 'float', 'log': True},
        "num_leaves" : {"low" : 10, "high" : 100, "step" : 5, 'type':'int'},
        'max_depth': {'low': 2, 'high': 10, 'type': 'int'},
        'min_child_samples': {'low': 1, 'high': 51, 'step': 5, 'type': 'int'},
        'colsample_bytree': {'low': 0.1, 'high': 1.0, 'type': 'float'},
        'reg_alpha': {'low': 0.0, 'high': 1.0, 'type': 'float'},
        'reg_lambda': {'low': 0.0, 'high': 1.0, 'type': 'float'},
        'n_estimators': {'low': 10, 'high': 100, 'step': 10, 'type': 'int'},
        "is_unbalance" : {"choices" : [True], 'type':'categorical'},
        "verbose" : {"choices" : [-1], 'type':'categorical'},
    },
    "MLPClassifier": {
        "hidden_layer_sizes" : {"choices" : [
            [128, 64, 32],
            [128, 64, 32, 16],
            [256, 128, 64, 32, 16],
        ], 'type':'categorical'},
        "alpha" : {'low': 0.0001, 'high': 0.01, 'type': 'float', 'log': True},
        "learning_rate" : {'choices': ['constant', 'invscaling', 'adaptive'], 'type':'categorical'},
        "learning_rate_init" : {'low': 0.001, 'high': 0.1, 'type': 'float', 'log': True},
        "early_stopping" : {'choices': [True], 'type':'categorical'},
        "max_iter" : {"choices" : [50], 'type':'categorical'},
    }
}

In [ ]:
study_logistic, model_logistic = optimize_model(LogisticRegression, param_spaces["LogisticRegression"], X_train, y_train, X_val , y_val, n_trials=100)
joblib.dump(model_logistic, "models/home_credit_logistic.joblib")

In [28]:
print("Score for Logistic Regression: ", study_logistic.best_value)
print(study_logistic.best_params)

Score for Logistic Regression:  0.7415298088164799
{'C': 2.90524273919139, 'max_iter': 1000, 'penalty': 'l2', 'class_weight': 'balanced'}


In [ ]:
study_rf, model_rf = optimize_model(RandomForestClassifier, param_spaces["RandomForestClassifier"], X_train, y_train, X_val , y_val, n_trials=100)
joblib.dump(model_rf, "models/home_credit_rf.joblib")

In [30]:
print("Score for Random Forest: ", study_rf.best_value)
print(study_rf.best_params)

Score for Random Forest:  0.7407392205308929
{'n_estimators': 90, 'max_depth': 10, 'criterion': 'gini', 'min_samples_leaf': 51, 'max_features': 0.3435949186368579, 'class_weight': 'balanced'}


In [ ]:
study_lgbm, model_lgbm = optimize_model(LGBMClassifier, param_spaces["LGBMClassifier"], X_train, y_train, X_val , y_val, n_trials=100)
joblib.dump(model_lgbm, "models/home_credit_lgbm.joblib")

In [39]:
print("Score for LGBM: ", study_lgbm.best_value)
print(study_lgbm.best_params)

Score for LGBM:  0.7519408632926617
{'learning_rate': 0.1525897078045497, 'num_leaves': 25, 'max_depth': 10, 'min_child_samples': 6, 'colsample_bytree': 0.39246210984663443, 'reg_alpha': 0.699615134448418, 'reg_lambda': 0.5409369253349579, 'n_estimators': 70, 'is_unbalance': True, 'verbose': -1}


In [ ]:
study_mlp, model_mlp = optimize_model(MLPClassifier, param_spaces["MLPClassifier"], X_train, y_train, X_val , y_val, n_trials=100)
joblib.dump(model_mlp, "models/home_credit_mlp.joblib")

In [41]:
print("Score for MLP: ", study_mlp.best_value)
print(study_mlp.best_params)

Score for MLP:  0.7433528703800805
{'hidden_layer_sizes': [128, 64, 32], 'alpha': 0.0003527051808306031, 'learning_rate': 'invscaling', 'learning_rate_init': 0.010035211915818264, 'early_stopping': True, 'max_iter': 50}


## Model evaluation

In [38]:
models = {
    "LogisticRegression" : joblib.load("models/german_logistic.joblib"),
    "MLP" : joblib.load("models/german_mlp.joblib"),
    "RandomForest" : joblib.load("models/german_rf.joblib"),
    "LightGBM" : joblib.load("models/german_lgbm.joblib"),
}
ks_threshold_dict = {}
models_dict = {}
for n, m in models.items():
    ks_threshold_dict[n] = ks_threshold(y_val, m.predict_proba(X_val)[:,1])
    models_dict[n] = [
        m,
        ks_threshold_dict[n]
    ]

Trying to unpickle estimator SimpleImputer from version 1.3.0 when using version 1.3.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
Trying to unpickle estimator StandardScaler from version 1.3.0 when using version 1.3.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
Trying to unpickle estimator Pipeline from version 1.3.0 when using version 1.3.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
Trying to unpickle estimator OneHotEncoder from version 1.3.0 when using version 1.3.1. This might lead to breaking code or invalid results. Use a

KeyError: "None of [Index(['ForeignWorker', 'Single', 'Age', 'LoanDuration', 'LoanAmount',\n       'LoanRateAsPercentOfIncome', 'YearsAtCurrentHome',\n       'NumberOfOtherLoansAtBank', 'NumberOfLiableIndividuals', 'HasTelephone',\n       'CheckingAccountBalance_geq_0', 'CheckingAccountBalance_geq_200',\n       'SavingsAccountBalance_geq_100', 'SavingsAccountBalance_geq_500',\n       'MissedPayments', 'NoCurrentLoan', 'CriticalAccountOrLoansElsewhere',\n       'OtherLoansAtBank', 'OtherLoansAtStore', 'HasCoapplicant',\n       'HasGuarantor', 'OwnsHouse', 'RentsHouse', 'Unemployed',\n       'YearsAtCurrentJob_lt_1', 'YearsAtCurrentJob_geq_4',\n       'JobClassIsSkilled'],\n      dtype='object')] are in the [columns]"

In [39]:
get_metrics(models_dict, X_train, y_train)

,AUC,Balanced Accuracy,Accuracy,Precision,Recall,F1,Brier Score


In [40]:
get_metrics(models_dict, X_val, y_val)

,AUC,Balanced Accuracy,Accuracy,Precision,Recall,F1,Brier Score


In [41]:
get_metrics(models_dict, X_test, y_test)

,AUC,Balanced Accuracy,Accuracy,Precision,Recall,F1,Brier Score


In [42]:
models_dict_fairness = {}
for n, m in models_dict.items():
    models_dict_fairness[n] = (m[0].predict_proba(X_test)[:,1] > m[1]).astype(int)

In [43]:
get_fairness_metrics(models_dict_fairness, y_test, A_test, 1)

,DPD,EOD,AOD,APVD,GMA


## Fairness

In [16]:
pipeline_preprocess = create_pipeline(
    X_train,
    y_train,
    lambda x : x,
)[:-1]
pipeline_preprocess.fit(X_train, y_train)
preprocess_column_names = pipeline_preprocess.get_feature_names_out()
X_train_preprocess = pipeline_preprocess.transform(X_train)
X_val_preprocess = pipeline_preprocess.transform(X_val)
X_test_preprocess = pipeline_preprocess.transform(X_test)

## Pre-processing (Reweighing / AIF360)

In [24]:
df_rw = pd.DataFrame(
    X_train_preprocess,
    columns = preprocess_column_names
)
df_rw["TARGET"] = y_train.values
x_train_aif = BinaryLabelDataset(
    df = df_rw,
    label_names = ["TARGET"],
    protected_attribute_names = ["cat__CODE_GENDER_F"]
)
rw = Reweighing(
    unprivileged_groups = [{"cat__CODE_GENDER_F": 1}],
    privileged_groups = [{"cat__CODE_GENDER_F": 0}],
)
rw.fit(x_train_aif)
rw_weights = rw.transform(x_train_aif).instance_weights

In [37]:
for n in ["LogisticRegression", "RandomForest", "LightGBM"]:
    best_params = models[n]["classifier"].get_params()
    rw_model = classifiers[n](**best_params)
    rw_model.fit(X_train_preprocess, y_train, sample_weight=rw_weights)
    rw_threshold = ks_threshold(y_val, rw_model.predict_proba(X_val_preprocess)[:,1])
    models_dict_fairness[n + "Reweighing"] = rw_model.predict_proba(X_test_preprocess)[:,1] > rw_threshold

TypeError: list indices must be integers or slices, not str

## In-processing

## EqualOpportunityClassifier

In [ ]:
gender_col_idx = np.where(preprocess_column_names == "cat__CODE_GENDER_F")[0][0]
eop_class = create_pipeline(
    X_train,
    y_train,
    EqualOpportunityClassifier(
        covariance_threshold = 0.1,
        sensitive_cols = gender_col_idx,
        positive_target = 1
    )
)
eop_class.fit(X_train, y_train)
eoq_ks_threshold = ks_threshold(y_val, eop_class.predict_proba(X_val)[:,1])
models_dict_fairness["EqualOpportunity"] = eop_class.predict_proba(X_test)[:,1] > eoq_ks_threshold
dop_class = create_pipeline(
    X_train,
    y_train,
    DemographicParityClassifier(
        covariance_threshold = 0.1,
        sensitive_cols = gender_col_idx,
    )
)
dop_class.fit(X_train, y_train)
dop_ks_threshold = ks_threshold(y_val, dop_class.predict_proba(X_val)[:,1])
models_dict_fairness["DemographicParity"] = dop_class.predict_proba(X_test)[:,1] > dop_ks_threshold

## FairGBM

In [ ]:
fairgbm_params = models["LightGBM"]["classifier"].get_params()
del fairgbm_params["objective"]
fairgbm = FairGBMClassifier(
    constraint_type="FNR",
    **fairgbm_params
)
fairgbm.fit(X_train_preprocess, y_train, constraint_group=A_train)
fairgbm_threshold = ks_threshold(y_val, fairgbm.predict_proba(X_val_preprocess)[:,1])
models_dict_fairness["FairGBM"] =fairgbm.predict_proba(X_test_preprocess)[:,1] > fairgbm_threshold

## Post-processing (Threshold-Optimizer / FairLearn)

In [ ]:
model_dict_thr_opt = {}
for name, model in models_dict.items():
    thr_opt = ThresholdOptimizer(
        estimator=model[0],
        constraints="equalized_odds",
        objective="balanced_accuracy_score",
        prefit=True,
        predict_method="predict_proba",
    )
    thr_opt.fit(X_train, y_train, sensitive_features=A_train)
    models_dict_fairness[name + "Thr.Opt."] = thr_opt.predict(X_test, sensitive_features=A_test)

## Fairness Evaluation

In [ ]:
df_metrics_results = get_fairness_metrics(
    models_dict_fairness,
    y_test,
    A_test,
    1
)
df_metrics_results_ = df_metrics_results.copy().reset_index()

In [ ]:
df_metrics_results_

In [ ]:
# Mapping each model to a unique color
color_mapping = {model: idx for idx, model in enumerate(df_metrics_results_["index"].unique())}

# Adjust fairness metrics to be "the higher the better" and scale them to be between 0 and 100
#for col in ['DPD', 'EOD', 'AOD', 'APVD']:
#    df_metrics_results_[col] = (1 - np.abs(df_metrics_results_[col])) * 100

# Scale the performance metrics to be between 0 and 100
#df_metrics_results_['GMA'] *= 100
#df_latest_adjusted['balanced_accuracy'] *= 100

# Generate the required number of colors from the Viridis colorscale
num_colors = len(color_mapping)
#colors = px.colors.qualitative.Dark2
colors = px.colors.sequential.Viridis
manual_colors = [colors[i * (len(colors) - 1) // (num_colors - 1)] for i in range(num_colors)]                                                              
# Create the Advanced Parallel Coordinates Plot with adjusted data
fig = go.Figure(data=
    go.Parcoords(
        line=dict(color=df_metrics_results_['index'].map(color_mapping),
                  showscale=False),
        dimensions=[
            dict(label='DPD', values=df_metrics_results_['DPD']),#, range=[65, 100]),
            dict(label='EOD', values=df_metrics_results_['EOD']),#, range=[65, 100]),
            dict(label='AOD', values=df_metrics_results_['AOD']),#, range=[65, 100]),
            dict(label='APVD', values=df_metrics_results_['APVD']),#, range=[65, 100]),
            dict(label='GMA', values=df_metrics_results_['GMA']),#, range=[65, 100])
            dict(label='balanced_accuracy', values=df_metrics_results_['balanced_accuracy'])#, range=[65, 100])
        ]
    )
)

# Add models to the legend using the dummy scatter plot approach and specified manual colors
for idx, (model, color) in enumerate(color_mapping.items()):
    fig.add_trace(
        go.Scatter(x=[None], y=[None],
                   mode='markers',
                   marker=dict(size=10, color=manual_colors[idx]),
                   name=model,
                   showlegend=True)
    )

# Update the layout and adjust the top margin
fig.update_layout(title='Parallel Coordinates Plot for Home Credit', margin=dict(t=100))

# Show the plot
fig.show()

## Explainability

**TODO**

*   Add easy way to include default parameters in optuna
*   Warnings on pipeline given numeric unique number of values
